<a href="https://colab.research.google.com/github/pevening/study_code/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B6%84%EC%84%9D_%EC%8A%A4%ED%84%B0%EB%94%94_5%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [34]:
# 탈퇴 회원과 지속회원 데이터를 바탕으로 탈퇴를 예측
# Decision Tree 활용한 지도학습의 분류 알고리즘

# 1. 데이터를 읽어 들이고 이용 데이터를 수정하자

In [61]:
customer = pd.read_csv('/content/gdrive/MyDrive/데이터분석 스터디/5장/customer_join.csv')
uselog_months = pd.read_csv('/content/gdrive/MyDrive/데이터분석 스터디/5장/use_log_months.csv')
customer 
uselog_months

,연월,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7
...,...,...,...
36837,201903,TS995853,8
36838,201903,TS998593,8
36839,201903,TS999079,3
36840,201903,TS999231,6


In [62]:
# 현재 달과 1개월 전의 이용 이력만으로 데이터 작성 -> 가입 개월 5개월 이내인 회원의 탈퇴 또한 예측 위해

year_months = list(uselog_months['연월'].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
  tmp = uselog_months.loc[uselog_months['연월']==year_months[i]]
  tmp.rename(columns={'count':'count_0'},inplace=True)
  tmp_before = uselog_months.loc[uselog_months['연월']==year_months[i-1]]
  del tmp_before['연월']
  tmp_before.rename(columns={'count':'count_1'},inplace=True)
  tmp = pd.merge(tmp,tmp_before,on='customer_id',how='left')
  uselog = pd.concat([uselog,tmp],ignore_index=True)
uselog

/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0
...,...,...,...,...
33846,201903,TS995853,8,11.0
33847,201903,TS998593,8,7.0
33848,201903,TS999079,3,2.0
33849,201903,TS999231,6,6.0


#2. 탈퇴 전월의 탈퇴 고객 데이터를 작성하자

In [63]:
# 탈퇴를 미연에 방지하는 것이 목적
# 월말까지 탈퇴 신청을 해야 다음 달에 탈퇴 되는 시스템
# 탈퇴 회원 추출 후 end_date의 1개월 전 계산

from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
exit_customer["연월"] = pd.to_datetime(exit_customer["exit_date"]).dt.strftime("%Y%m")
uselog["연월"] = uselog["연월"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "연월"], how="left")
print(len(uselog))
exit_uselog

# 책 코드 틀림

# 탈퇴한 회원의 탈퇴 전월 데이터뿐 / 결측치 많음

<ipython-input-63-6dac9d31d462>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["exit_date"] = None
<ipython-input-63-6dac9d31d462>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
/usr/local/lib/python3.9/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

33851


<ipython-input-63-6dac9d31d462>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer["연월"] = pd.to_datetime(exit_customer["exit_date"]).dt.strftime("%Y%m")


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33846,201903,TS995853,8,11.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33847,201903,TS998593,8,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33848,201903,TS999079,3,2.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33849,201903,TS999231,6,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
exit_uselog = exit_uselog.dropna(subset=['name'])
# subset 사용하여 특정 열이 nan인 값 삭제
print(len(exit_uselog))
print(len(exit_uselog['customer_id'].unique()))
exit_uselog

1104
1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30758,201902,TS645212,4,2.0,XXXX,C03,F,2018-03-01,2019-03-31,CA1,...,6000.0,일반,4.500000,4.5,7.0,1.0,0.0,2019-03-31,12.0,2019-02-28 00:00:00
30787,201902,TS741703,5,6.0,XXXX,C03,M,2018-12-08,2019-03-31,CA3,...,6000.0,입회비무료,6.250000,6.0,8.0,5.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00
30827,201902,TS859258,1,3.0,XXXXX,C02,F,2018-12-07,2019-03-31,CA3,...,7500.0,입회비무료,2.500000,2.0,5.0,1.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00
30842,201902,TS886985,5,3.0,XXX,C02,F,2018-03-01,2019-03-31,CA1,...,7500.0,일반,4.250000,4.0,7.0,2.0,1.0,2019-03-31,12.0,2019-02-28 00:00:00


#3. 지속 회원의 데이터를 작성하자

In [65]:
# 지속 회원 추출 후 uselog 데이터에 결합

conti_customer = customer.loc[customer['is_deleted']==0]
conti_uselog = pd.merge(uselog,conti_customer, on=['customer_id'],how='left')
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=['name'])
print(len(conti_uselog))

# 탈퇴 회원의 데이터는 1104개인데 비해 지속회원 데이터 수는 27422로 차이가 많이 남 -> 데이터 불균형


33851
27422


In [66]:
# 회원 당 여러번 들어가 있는 지속데이터를 회원 당 1번씩 들어가도록 언더샘플링
# 데이터를 섞고 중복 제거
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset='customer_id')
conti_uselog

# 데이터 개수가 2842개까지 줄어듦
# 랜덤으로 1개씩 들어가도록 해서 데이터 달라질수도

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201903,OA236156,6,3.0,XXXXX,C01,F,2015-06-01,NaN,CA1,...,종일,10500.0,일반,4.083333,3.5,8.0,2.0,1.0,2019-04-30,46.0
1,201806,HI153809,7,5.0,XXXX,C02,M,2016-03-01,NaN,CA1,...,주간,7500.0,일반,4.666667,4.5,8.0,2.0,1.0,2019-04-30,37.0
2,201809,GD030141,9,NaN,XX,C01,M,2018-09-01,NaN,CA1,...,종일,10500.0,일반,8.142857,8.0,10.0,5.0,1.0,2019-04-30,7.0
3,201810,HI072340,6,9.0,XXXXX,C02,F,2015-08-01,NaN,CA1,...,주간,7500.0,일반,5.083333,4.5,9.0,3.0,1.0,2019-04-30,44.0
4,201902,PL351843,1,1.0,XXXXX,C01,M,2015-11-01,NaN,CA1,...,종일,10500.0,일반,5.166667,5.5,9.0,1.0,1.0,2019-04-30,41.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25778,201903,TS926301,8,NaN,XXXXXX,C02,F,2019-03-08,NaN,CA1,...,주간,7500.0,일반,8.000000,8.0,8.0,8.0,1.0,2019-04-30,1.0
26110,201903,GD748823,6,NaN,XXXX,C02,F,2019-03-04,NaN,CA1,...,주간,7500.0,일반,6.000000,6.0,6.0,6.0,1.0,2019-04-30,1.0
26615,201903,GD485275,11,NaN,XXXXX,C03,M,2019-03-09,NaN,CA1,...,야간,6000.0,일반,11.000000,11.0,11.0,11.0,0.0,2019-04-30,1.0
26684,201903,IK726211,9,NaN,XXX,C02,F,2019-03-01,NaN,CA1,...,주간,7500.0,일반,9.000000,9.0,9.0,9.0,1.0,2019-04-30,1.0


In [67]:
# 지속 회원 데이터와 탈퇴 회원 데이터 새로 결합

predict_data = pd.concat([conti_uselog,exit_uselog],ignore_index=True)
predict_data

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201903,OA236156,6,3.0,XXXXX,C01,F,2015-06-01,NaN,CA1,...,10500.0,일반,4.083333,3.5,8.0,2.0,1.0,2019-04-30,46.0,NaN
1,201806,HI153809,7,5.0,XXXX,C02,M,2016-03-01,NaN,CA1,...,7500.0,일반,4.666667,4.5,8.0,2.0,1.0,2019-04-30,37.0,NaN
2,201809,GD030141,9,NaN,XX,C01,M,2018-09-01,NaN,CA1,...,10500.0,일반,8.142857,8.0,10.0,5.0,1.0,2019-04-30,7.0,NaN
3,201810,HI072340,6,9.0,XXXXX,C02,F,2015-08-01,NaN,CA1,...,7500.0,일반,5.083333,4.5,9.0,3.0,1.0,2019-04-30,44.0,NaN
4,201902,PL351843,1,1.0,XXXXX,C01,M,2015-11-01,NaN,CA1,...,10500.0,일반,5.166667,5.5,9.0,1.0,1.0,2019-04-30,41.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3941,201902,TS645212,4,2.0,XXXX,C03,F,2018-03-01,2019-03-31 00:00:00,CA1,...,6000.0,일반,4.500000,4.5,7.0,1.0,0.0,2019-03-31,12.0,2019-02-28 00:00:00
3942,201902,TS741703,5,6.0,XXXX,C03,M,2018-12-08,2019-03-31 00:00:00,CA3,...,6000.0,입회비무료,6.250000,6.0,8.0,5.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00
3943,201902,TS859258,1,3.0,XXXXX,C02,F,2018-12-07,2019-03-31 00:00:00,CA3,...,7500.0,입회비무료,2.500000,2.0,5.0,1.0,0.0,2019-03-31,3.0,2019-02-28 00:00:00
3944,201902,TS886985,5,3.0,XXX,C02,F,2018-03-01,2019-03-31 00:00:00,CA1,...,7500.0,일반,4.250000,4.0,7.0,2.0,1.0,2019-03-31,12.0,2019-02-28 00:00:00


#4. 예측할 달의 재적 기간을 작성하자

In [68]:
# 재적 기간 열을 추가

# predict_data['period'] = 0
# predict_data['now_date'] = pd.to_datetime(predict_data['연월'],format='%Y%m')
# predict_data['start_date'] = pd.to_datetime(predict_data['start_date'])
# for i in range(len(predict_data)):
#   delta = relativedelta(predict_data['now_date'][i],predict_data['start_date'][i])
#   predict_data['period'][i] = int(delta.years*12 + delta.months)
# predict_data

predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["연월"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)
predict_data.head()

<ipython-input-68-ad9bb7070d3f>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_data["period"][i] = int(delta.years*12 + delta.months)


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201903,OA236156,6,3.0,XXXXX,C01,F,2015-06-01,NaN,CA1,...,4.083333,3.5,8.0,2.0,1.0,2019-04-30,46.0,NaN,45,2019-03-01
1,201806,HI153809,7,5.0,XXXX,C02,M,2016-03-01,NaN,CA1,...,4.666667,4.5,8.0,2.0,1.0,2019-04-30,37.0,NaN,27,2018-06-01
2,201809,GD030141,9,NaN,XX,C01,M,2018-09-01,NaN,CA1,...,8.142857,8.0,10.0,5.0,1.0,2019-04-30,7.0,NaN,0,2018-09-01
3,201810,HI072340,6,9.0,XXXXX,C02,F,2015-08-01,NaN,CA1,...,5.083333,4.5,9.0,3.0,1.0,2019-04-30,44.0,NaN,38,2018-10-01
4,201902,PL351843,1,1.0,XXXXX,C01,M,2015-11-01,NaN,CA1,...,5.166667,5.5,9.0,1.0,1.0,2019-04-30,41.0,NaN,39,2019-02-01


#5. 결측치를 제거하자

In [69]:
predict_data.isna().sum()

# count_1 에 결측치만 제거

연월                      0
customer_id             0
count_0                 0
count_1               260
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [70]:
predict_data = predict_data.dropna(subset=['count_1'])
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2634
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2634
period                  0
now_date                0
dtype: int64

#6. 문자형 변수를 처리할 수 있게 가공하자

In [71]:
# 범주형 즉 카테고리 변수를 처리하자

target_col = ['campaign_name','class_name','gender','count_1','routine_flg','period','is_deleted']
predict_data = predict_data[target_col]
predict_data

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,일반,종일,F,3.0,1.0,45,0.0
1,일반,주간,M,5.0,1.0,27,0.0
3,일반,주간,F,9.0,1.0,38,0.0
4,일반,종일,M,1.0,1.0,39,0.0
5,일반,주간,F,6.0,1.0,32,0.0
...,...,...,...,...,...,...,...
3941,일반,야간,F,2.0,0.0,11,1.0
3942,입회비무료,야간,M,6.0,0.0,1,1.0
3943,입회비무료,주간,F,3.0,0.0,1,1.0
3944,일반,주간,F,3.0,1.0,11,1.0


In [72]:
# 더미 변수 생성

predict_data = pd.get_dummies(predict_data)
predict_data

,count_1,routine_flg,period,is_deleted,campaign_name_일반,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_야간,class_name_종일,class_name_주간,gender_F,gender_M
0,3.0,1.0,45,0.0,1,0,0,0,1,0,1,0
1,5.0,1.0,27,0.0,1,0,0,0,0,1,0,1
3,9.0,1.0,38,0.0,1,0,0,0,0,1,1,0
4,1.0,1.0,39,0.0,1,0,0,0,1,0,0,1
5,6.0,1.0,32,0.0,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3941,2.0,0.0,11,1.0,1,0,0,1,0,0,1,0
3942,6.0,0.0,1,1.0,0,1,0,1,0,0,0,1
3943,3.0,0.0,1,1.0,0,1,0,0,0,1,1,0
3944,3.0,1.0,11,1.0,1,0,0,0,0,1,1,0


In [73]:
# 각 더미 변수마다 하나씩 지워준다 - 다중공산성 방지

del predict_data['campaign_name_일반']
del predict_data['class_name_야간']
del predict_data['gender_M']
predict_data

,count_1,routine_flg,period,is_deleted,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_종일,class_name_주간,gender_F
0,3.0,1.0,45,0.0,0,0,1,0,1
1,5.0,1.0,27,0.0,0,0,0,1,0
3,9.0,1.0,38,0.0,0,0,0,1,1
4,1.0,1.0,39,0.0,0,0,1,0,0
5,6.0,1.0,32,0.0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...
3941,2.0,0.0,11,1.0,0,0,0,0,1
3942,6.0,0.0,1,1.0,1,0,0,0,0
3943,3.0,0.0,1,1.0,1,0,0,1,1
3944,3.0,1.0,11,1.0,0,0,0,1,1


#7. 의사결정 트리를 사용해서 탈퇴 예측 모델을 구축하자

In [74]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data['is_deleted']==1]
conti = predict_data.loc[predict_data['is_deleted']==0].sample(len(exit))
# exit 와 conti 개수 맞춰주기 위해 랜덤으로 exit 개수만큼 conti 에서 뽑음

X = pd.concat([exit,conti],ignore_index=True)
y = X['is_deleted']
del X['is_deleted']

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)
model = DecisionTreeClassifier(random_state=0)
# random_state 는 특정 난수 부여하는 것. 같은 번호는 같은 결과 도출한다고 생각
model.fit(X_train,y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1.
 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1.
 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0.
 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1.
 1. 1. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0.
 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0.
 1. 1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1.
 1. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0.
 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0.
 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0.
 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 1.

In [75]:
# 실제 정답과 예측하여 분류한 값 비교 

results_test = pd.DataFrame({'y_test':y_test,'y_pred':y_test_pred})
results_test

,y_test,y_pred
1015,1.0,1.0
7,1.0,1.0
1293,0.0,1.0
1304,0.0,0.0
1266,0.0,0.0
...,...,...
850,1.0,0.0
1902,0.0,0.0
1022,1.0,0.0
1198,0.0,0.0


#8. 예측 모델을 평가하고 모델을 튜닝해 보자

In [76]:
# 정답률을 계산해보자

correct = len(results_test.loc[results_test['y_test']==results_test['y_pred']])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

# 학습용 데이터의 score 는 약 90%
# 학습용 데이터와 평가용 데이터 예측 정확도 차이가 작은 것이 이상적

0.8954372623574145


In [77]:
print(model.score(X_test,y_test))
print(model.score(X_train,y_train))

# 학습용 데이터가 평가용 데이터보다 정확도 훨씬 높음. 학습용 데이터에 너무 맞춘 과적합 경향

0.8954372623574145
0.9740177439797212


In [78]:
# 모델 파라미터 조정해 과적합 해결해보자 !

X = pd.concat([exit,conti],ignore_index=True)
y = X['is_deleted']
del X['is_deleted']
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0,max_depth=5)
# max_depth 를 활용해 트리 깊이 limit 줘봄
model.fit(X_train,y_train)

print(model.score(X_test,y_test))
print(model.score(X_train,y_train))

# train 데이터와 test 데이터 성능 정확도가 거의 일치하는 모습 !!
# 모델 단순화 장점 -> 어떤 데이터가 와도 대응할 수 있는 좋은 모델 만들어짐


0.903041825095057
0.9309252217997465


#9. 모델에 기여하는 변수를 확인하자

In [79]:
# 변수의 기여율을 알아보자

importance = pd.DataFrame({'feature_names':X.columns, "coefficient": model.feature_importances_})
importance

# 1개월전 이용 횟수, 기간, 정기 이용 여부가 높은 기여

,feature_names,coefficient
0,count_1,0.365611
1,routine_flg,0.128536
2,period,0.497077
3,campaign_name_입회비무료,0.000000
4,campaign_name_입회비반액할인,0.004092
5,class_name_종일,0.000278
6,class_name_주간,0.000000
7,gender_F,0.004406


#10. 회원 탈퇴를 예측하자

In [82]:
# 예측해볼 데이터 작성

count_1 = 7
routine_flg = 1
period = 20
campaign_name = '입회비무료'
class_name = '종일'
gender = 'M'


In [83]:
# 모델에 넣기 위해 데이터 가공

if campaign_name == "입회비반값할인":
    campaign_name_list = [1, 0]
elif campaign_name == "입회비무료":
    campaign_name_list = [0, 1]
elif campaign_name == "일반":
    campaign_name_list = [0, 0]
if class_name == "종일":
    class_name_list = [1, 0]
elif class_name == "주간":
    class_name_list = [0, 1]
elif class_name == "야간":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)

print(model.predict([input_data]))
print(model.predict_proba([input_data]))

# 결과
# -> 모델은 이 회원에 대해 1 즉 탈퇴를 예상.
# 0의 확률은 약 0.015, 1의 확률은 약 0.986

# 숫자 바꿔서 해보면 결과 달라짐

[0.]
[[0.9382716 0.0617284]]


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
